<a href="https://colab.research.google.com/github/ViniciusJose05/anime/blob/main/anime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de Dados em Séries de Anime:  Gêneros, Avaliações e Tendências

### Equipe:
* Maria Júlia Silva Fonseca Guedes Nobre
* Vinícius José Aragão de Almeida Barrozo

### Objetivos:
- Identificar os gêneros mais frequentes.
- Avaliar quais gêneros têm as melhores (e piores) notas médias.
- Descobrir as combinações de gêneros mais comuns.
- Analisar a relação entre popularidade e avaliação.
- Testar um modelo preditivo baseado nas avaliações.

# 1. Inicialização da Database

Download dos arquivos via API do kaggle e criação do dataframe inicial

In [ ]:
import kagglehub
import polars as pl

# Baixar e carregar o dataset
path = kagglehub.dataset_download("hernan4444/anime-recommendation-database-2020")
anime_polars = pl.read_csv(path + "/anime.csv", null_values="Unknown")

anime_polars.head()

100%|██████████| 661M/661M [00:09<00:00, 73.7MB/s]

Extracting files...


MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
i64,str,f64,str,str,str,str,i64,str,str,str,str,str,str,str,str,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,"""Cowboy Bebop""",8.78,"""Action, Adventure, Comedy, Dra…","""Cowboy Bebop""","""カウボーイビバップ""","""TV""",26,"""Apr 3, 1998 to Apr 24, 1999""","""Spring 1998""","""Bandai Visual""","""Funimation, Bandai Entertainme…","""Sunrise""","""Original""","""24 min. per ep.""","""R - 17+ (violence & profanity)""",28.0,39,1251960,61971,105808,718161,71513,26678,329800,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
5,"""Cowboy Bebop: Tengoku no Tobir…",8.39,"""Action, Drama, Mystery, Sci-Fi…","""Cowboy Bebop:The Movie""","""カウボーイビバップ 天国の扉""","""Movie""",1,"""Sep 1, 2001""",null,"""Sunrise, Bandai Visual""","""Sony Pictures Entertainment""","""Bones""","""Original""","""1 hr. 55 min.""","""R - 17+ (violence & profanity)""",159.0,518,273145,1174,4143,208333,1935,770,57964,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
6,"""Trigun""",8.24,"""Action, Sci-Fi, Adventure, Com…","""Trigun""","""トライガン""","""TV""",26,"""Apr 1, 1998 to Sep 30, 1998""","""Spring 1998""","""Victor Entertainment""","""Funimation, Geneon Entertainme…","""Madhouse""","""Manga""","""24 min. per ep.""","""PG-13 - Teens 13 or older""",266.0,201,558913,12944,29113,343492,25465,13925,146918,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
7,"""Witch Hunter Robin""",7.27,"""Action, Mystery, Police, Super…","""Witch Hunter Robin""","""Witch Hunter ROBIN (ウイッチハンターロビ…","""TV""",26,"""Jul 2, 2002 to Dec 24, 2002""","""Summer 2002""","""TV Tokyo, Bandai Visual, Dents…","""Funimation, Bandai Entertainme…","""Sunrise""","""Original""","""25 min. per ep.""","""PG-13 - Teens 13 or older""",2481.0,1467,94683,587,4300,46165,5121,5378,33719,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
8,"""Bouken Ou Beet""",6.98,"""Adventure, Fantasy, Shounen, S…","""Beet the Vandel Buster""","""冒険王ビィト""","""TV""",52,"""Sep 30, 2004 to Sep 29, 2005""","""Fall 2004""","""TV Tokyo, Dentsu""",null,"""Toei Animation""","""Manga""","""23 min. per ep.""","""PG - Children""",3710.0,4369,13224,18,642,7314,766,1108,3394,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


# 2. Pré Processamento

###  2.1. Remover valores nulos em colunas importantes (`Genres` e `Scores`)

In [ ]:
df_clean = anime_polars.filter(
    (anime_polars['Genres'].is_not_null()) & (anime_polars['Score'].is_not_null())
)

print(df_clean)

shape: (12_406, 35)
┌────────┬──────────────────┬───────┬──────────────────┬───┬─────────┬─────────┬─────────┬─────────┐
│ MAL_ID ┆ Name             ┆ Score ┆ Genres           ┆ … ┆ Score-4 ┆ Score-3 ┆ Score-2 ┆ Score-1 │
│ ---    ┆ ---              ┆ ---   ┆ ---              ┆   ┆ ---     ┆ ---     ┆ ---     ┆ ---     │
│ i64    ┆ str              ┆ f64   ┆ str              ┆   ┆ f64     ┆ f64     ┆ f64     ┆ f64     │
╞════════╪══════════════════╪═══════╪══════════════════╪═══╪═════════╪═════════╪═════════╪═════════╡
│ 1      ┆ Cowboy Bebop     ┆ 8.78  ┆ Action,          ┆ … ┆ 3184.0  ┆ 1357.0  ┆ 741.0   ┆ 1580.0  │
│        ┆                  ┆       ┆ Adventure,       ┆   ┆         ┆         ┆         ┆         │
│        ┆                  ┆       ┆ Comedy, Dra…     ┆   ┆         ┆         ┆         ┆         │
│ 5      ┆ Cowboy Bebop:    ┆ 8.39  ┆ Action, Drama,   ┆ … ┆ 577.0   ┆ 221.0   ┆ 109.0   ┆ 379.0   │
│        ┆ Tengoku no       ┆       ┆ Mystery, Sci-Fi… ┆   ┆         ┆ 

### 2.2. Separar a coluna `Genres` em uma lista

In [ ]:
df_clean = df_clean.with_columns(
    pl.col('Genres')
    .str.replace("Hentai", "Adult Content")
    .str.split(", ")                       # converte para lista
)

print(df_clean)

shape: (12_406, 35)
┌────────┬──────────────────┬───────┬──────────────────┬───┬─────────┬─────────┬─────────┬─────────┐
│ MAL_ID ┆ Name             ┆ Score ┆ Genres           ┆ … ┆ Score-4 ┆ Score-3 ┆ Score-2 ┆ Score-1 │
│ ---    ┆ ---              ┆ ---   ┆ ---              ┆   ┆ ---     ┆ ---     ┆ ---     ┆ ---     │
│ i64    ┆ str              ┆ f64   ┆ list[str]        ┆   ┆ f64     ┆ f64     ┆ f64     ┆ f64     │
╞════════╪══════════════════╪═══════╪══════════════════╪═══╪═════════╪═════════╪═════════╪═════════╡
│ 1      ┆ Cowboy Bebop     ┆ 8.78  ┆ ["Action",       ┆ … ┆ 3184.0  ┆ 1357.0  ┆ 741.0   ┆ 1580.0  │
│        ┆                  ┆       ┆ "Adventure", …   ┆   ┆         ┆         ┆         ┆         │
│        ┆                  ┆       ┆ "Spa…            ┆   ┆         ┆         ┆         ┆         │
│ 5      ┆ Cowboy Bebop:    ┆ 8.39  ┆ ["Action",       ┆ … ┆ 577.0   ┆ 221.0   ┆ 109.0   ┆ 379.0   │
│        ┆ Tengoku no       ┆       ┆ "Drama", …       ┆   ┆         ┆ 

###  2.3. Criar coluna com combinação ordenada dos gêneros

In [ ]:
def process_genres(genres):
    try:
        if genres is not None and len(genres) > 0:
            return ", ".join(sorted(genres))
        else:
            return ""
    except (TypeError, AttributeError):
        return ""

df_clean = df_clean.with_columns(
    pl.col('Genres').map_elements(process_genres, return_dtype=pl.Utf8).alias('Genres_combination')
)
# download em formato json da database polars
df_clean.write_json('df_clean.json')


# 3. Análises Exploratórias

In [ ]:
import plotly.express as px

# Explodir para análise individual de gêneros
df_exploded = df_clean.explode('Genres')

print(df_clean)

shape: (12_406, 36)
┌────────┬───────────────┬───────┬───────────────┬───┬─────────┬─────────┬─────────┬───────────────┐
│ MAL_ID ┆ Name          ┆ Score ┆ Genres        ┆ … ┆ Score-3 ┆ Score-2 ┆ Score-1 ┆ Genres_combin │
│ ---    ┆ ---           ┆ ---   ┆ ---           ┆   ┆ ---     ┆ ---     ┆ ---     ┆ ation         │
│ i64    ┆ str           ┆ f64   ┆ list[str]     ┆   ┆ f64     ┆ f64     ┆ f64     ┆ ---           │
│        ┆               ┆       ┆               ┆   ┆         ┆         ┆         ┆ str           │
╞════════╪═══════════════╪═══════╪═══════════════╪═══╪═════════╪═════════╪═════════╪═══════════════╡
│ 1      ┆ Cowboy Bebop  ┆ 8.78  ┆ ["Action",    ┆ … ┆ 1357.0  ┆ 741.0   ┆ 1580.0  ┆ Action,       │
│        ┆               ┆       ┆ "Adventure",  ┆   ┆         ┆         ┆         ┆ Adventure,    │
│        ┆               ┆       ┆ … "Spa…       ┆   ┆         ┆         ┆         ┆ Comedy, Dra…  │
│ 5      ┆ Cowboy Bebop: ┆ 8.39  ┆ ["Action",    ┆ … ┆ 221.0   ┆ 109.0 

### 3.1. Gêneros mais frequentes


In [ ]:
genero_freq = (
    df_exploded.group_by('Genres')
    .len()
    .sort('len', descending=True)
    .rename({'len': 'Frequencia'})
)

px.bar(genero_freq.to_pandas(), x='Genres', y='Frequencia', title='Gêneros Mais Frequentes').show()

### 3.2. Nota média por gênero

In [ ]:
genero_score = (
    df_exploded.group_by('Genres')
    .agg(pl.col('Score').mean().alias('Nota Média'))
    .sort('Nota Média', descending=True)
)

px.bar(genero_score.head(15).to_pandas(), x='Genres', y='Nota Média', title='Top 15 Gêneros com Melhores Notas').show()
px.bar(genero_score.tail(15).to_pandas(), x='Genres', y='Nota Média', title='15 Gêneros com Piores Notas').show()

### 3.3. Combinações de gêneros mais comuns

In [ ]:
df_combos = df_clean.filter(pl.col('Genres').list.len() > 1)

combo_freq = (
    df_combos
    .group_by('Genres_combination')
    .agg(pl.count().alias('Frequencia'))
    .sort('Frequencia', descending=True)
)

px.bar(combo_freq.head(15).to_pandas(), x='Genres_combination', y='Frequencia',
       title='🔗 Combinações de Gêneros Mais Comuns (com 2 ou mais gêneros)').show()

/tmp/ipython-input-8-1626674012.py:6: DeprecationWarning:

`pl.count()` is deprecated. Please use `pl.len()` instead.



### 3.4. Estúdios com melhor nota média


In [ ]:
df_studios = df_clean.filter(
    (pl.col('Studios').is_not_null()) & (pl.col('Studios') != "None")
)

# Agrupar por estúdio e calcular média e contagem
studio_avg = (
    df_studios
    .group_by('Studios')
    .agg([
        pl.col('Score').mean().alias('Nota Média'),
        pl.count().alias('Quantidade de Animes')
    ])
    .filter(pl.col('Quantidade de Animes') >= 5)
    .sort('Nota Média', descending=True)
)

fig = px.bar(
    studio_avg.head(15).to_pandas(),  # Top 15 estúdios com melhor nota média
    x='Studios',
    y='Nota Média',
    title='🎬 Estúdios com as Melhores Notas Médias (com pelo menos 5 animes)',
    text='Nota Média',
    labels={'Studios': 'Estúdio', 'Nota Média': 'Nota Média'}
)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

/tmp/ipython-input-9-2758623128.py:11: DeprecationWarning:

`pl.count()` is deprecated. Please use `pl.len()` instead.



 ### 3.5. Relação entre popularidade e avaliação

In [ ]:
relacao_popularidade = df_clean.select(['Score', 'Members', 'Genres_combination']).to_pandas()

px.scatter(relacao_popularidade, x='Score', y='Members', color='Genres_combination',
           size='Members', hover_data=['Genres_combination'],
           title='Relação entre Nota e Popularidade por Gênero').show()


# 4. Modelo SVM para predição de notas



### 4.1. Filtragem das colunas necessárias

In [ ]:
df_clean.columns

['MAL_ID',
 'Name',
 'Score',
 'Genres',
 'English name',
 'Japanese name',
 'Type',
 'Episodes',
 'Aired',
 'Premiered',
 'Producers',
 'Licensors',
 'Studios',
 'Source',
 'Duration',
 'Rating',
 'Ranked',
 'Popularity',
 'Members',
 'Favorites',
 'Watching',
 'Completed',
 'On-Hold',
 'Dropped',
 'Plan to Watch',
 'Score-10',
 'Score-9',
 'Score-8',
 'Score-7',
 'Score-6',
 'Score-5',
 'Score-4',
 'Score-3',
 'Score-2',
 'Score-1',
 'Genres_combination']

In [ ]:
df_para_ml = df_clean.select(
    ['MAL_ID','Name', 'Genres', 'Score']
)

df_para_ml_com_membros = df_clean.select(
    ['MAL_ID','Name', 'Genres', 'Score', 'Members']
)

### 4.2. One-Hot Encoding nos generos das listas de cada linha

Uso de Machine Learning utilizando a database a fim de validar as transformações realizadas e uso da mesma

In [ ]:
def one_hot_encode(df, column_name):
    # Get all unique genres
    all_genres = df[column_name].explode().unique().sort()

    expressions = []
    for genre in all_genres:
        expressions.append(
            pl.col(column_name).list.contains(genre).alias(f"{column_name}_{genre}")
        )

    df = df.with_columns(expressions)
    return df.drop(column_name)

df_para_ml = one_hot_encode(df_para_ml, 'Genres')
df_para_ml = df_para_ml.drop('Name', 'MAL_ID')

df_para_ml_com_membros = one_hot_encode(df_para_ml_com_membros, 'Genres')
df_para_ml_com_membros = df_para_ml_com_membros.drop('Name', 'MAL_ID')

df_para_ml.head(), df_para_ml_com_membros.head()

# download dos dataframes polars em csv
df_para_ml.write_csv('df_para_ml.csv')
df_para_ml_com_membros.write_csv('df_para_ml_com_membros.csv')

### 4.3. Função de treinamento e teste

Implementação de modelo de SVM utilizando 10-10-fold para predição da nota baseado em generos, contendo ou não o numero de membros de cada entrada.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

def train_and_evaluate_knn(df, n_neighbors=5):
    # Separate features (X) and target (y)
    # Assuming 'Score' is the target variable and the rest are features
    X = df.drop('Score').to_numpy()
    y = df['Score'].to_numpy()

    # Define the number of folds
    n_folds = 10

    # Initialize KFold
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    # Initialize list to store evaluation scores
    mse_scores = []

    # Loop through each fold
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Initialize the KNeighborsRegressor model
        model = KNeighborsRegressor(n_neighbors=n_neighbors)

        # Train the model
        model.fit(X_train, y_train)

        # Predict on the test set
        y_pred = model.predict(X_test)

        # Evaluate the model using Mean Squared Error
        mse = mean_squared_error(y_test, y_pred)
        mse_scores.append(mse)

    # Calculate the average MSE across all folds
    average_mse = np.mean(mse_scores)

    return average_mse, mse_scores

### 4.4. Resultados do modelo

4.4.1. Modelo somente com Score e generos

In [ ]:
average_mse, mse_scores = train_and_evaluate_knn(df_para_ml)
print("MSE por fold:", mse_scores)
print("Média do erro quadrático médio:", average_mse)

MSE por fold: [0.5544576470588236, 0.6037359935535859, 0.5378227365028203, 0.5592274713940371, 0.5833630362610798, 0.5617809282836422, 0.5893821451612905, 0.5865917161290323, 0.5844107032258063, 0.5785915580645161]
Média do erro quadrático médio: 0.5739363935634635


4.4.2. Modelo com número de membros

In [ ]:
average_mse_membros, mse_scores_membros = train_and_evaluate_knn(df_para_ml_com_membros)
print(f"MSE por fold: {mse_scores_membros}")
print(f"Média do erro quadrático médio: {average_mse_membros}")

MSE por fold: [0.465828425463336, 0.4880958904109589, 0.4557119452054795, 0.4517028622078969, 0.47075634165995167, 0.4637664174053183, 0.4921363838709677, 0.4465764516129034, 0.4988253741935484, 0.49148840322580645]
Média do erro quadrático médio: 0.47248884952561665
